# Enzymes

In [1]:
import pandas as pd
import numpy as np
import re

#from tensorflow.keras.models import Sequential
#from tensorflow.keras.layers import Dense

#from pycaret.classification import *

### Remettre tout en ordre

In [2]:
""" Ouverture du document """

df = pd.read_csv('dataset_enzymes.csv', encoding='ISO-8859-1', delimiter=';')

df = df.drop(['Unnamed: 8'], axis=1) # Cette ligne contient des annotations inutiles
df = df.drop(['STOP'], axis=1) # On enlève la ligne de STOP
df = df.drop([0, 1, 2, 5, 7]) # Il manque trop d'infos ou lignes vides

for i in range(7):
    df = df.drop([f'Unnamed: {i}'], axis=1) # Que des NaN
    
    
""" Rename les colonnes et les lignes """

for i in range(13,31):
    df = df.rename(columns={f'Unnamed: {i}': df[f'Unnamed: {i}'][6]})
for j in [7, 9, 10, 11, 12]:
    df = df.rename(columns={f'Unnamed: {j}': df[f'Unnamed: {j}'][14]})
    
df = df.rename(index={3: 'Nom', 4: 'Pureté', 6:'Abréviation'})
for k in range(8, 13):
    df = df.rename(index={k: df['Acc. Number'][k]})
for h in range(16, 468):
    df = df.rename(index={h: df['Acc. Number'][h]})

for l in range(468, 904):
    df = df.drop([l])
df = df.drop(['Acc. Number'], axis=1) # On a déjà rentré les infos
df = df.drop([13, 14, 15]) # On a déjà rentré les infos
 

""" Enlever les colonnes inutiles """
    
    # On enlève la ligne qu'on ne connait pas 
df = df.drop(['Q5N2G0'])

# S'il y a - et -, on enlève la ligne
for nom in df.index:
    if df['Induction (Gel)'][nom] == '-' and df['Expression (Gel)'][nom] == '-':
        df = df.drop([nom])

# Maintenant on enlève le reste des données
#df = df.drop(df.columns[:4], axis=1)
#df = df.drop(df.index[:8])
        
name_lines = df.index
name_columns = df.columns

# Si l'une des valeurs est négative ou nan, on remplace par 0
for i in range(len(name_columns)):
    for j in range(len(name_lines)):
        cell = df[name_columns[i]][name_lines[j]]
        # On remplace les , par des .
        cell = str(cell).replace(',', '.')
        try:
            if float(cell) < 0:
                cell = 0
        except:
            pass

# On stocke le bradford
bradford = {}
for line in range(8, len(df.index)):
    bradford[df.index[line]] = float(df['Bradford (mg/mL)'][line].replace(',', '.'))

# Puis on enlève les colonnes inutilisables
df = df.drop(['TAM Ref.', 'Bradford (mg/mL)', 'Induction (Gel)', 'Expression (Gel)'], axis=1)
    
# Et les puretés
purities = {}
for row in range(len(df.columns)):
    if df[df.columns[row]]['Pureté'] !=  '---':
        purities[df.columns[row]] = int(df[df.columns[row]]['Pureté'][:-1])
    else:
        purities[df.columns[row]] = 100

# Et les lignes
for line in range(8):
    df = df.drop([df.index[0]])
        
df.head(10)

,2OATA,Pyr,2OPAA,2OHAA,4MT2OB,MOPA,tMePyr,2O2PAA,2O4PBA,HPA,3H2OBA,BOHiPr,BOHPh,4H2OA,4H4PA,2KDGA,"3,3diMe",NoCeto
C7RAM1,"0,80","0,70","0,03","0,10","0,12","0,02","0,02","0,01","0,06","0,00","0,02","0,01","0,01","0,09","0,36","0,01","0,01","0,01"
P44336,"2,72","0,52","0,13","0,10","0,35","0,03","0,01","0,01","0,07","-0,01","0,03","0,01","0,01","0,14","0,26","0,01","0,01","0,01"
Q7VLP0,"1,10","0,91","0,24","0,21","0,39","0,06","0,01","0,01","0,08","0,00","0,03","0,02","0,02","0,19","0,54","0,01","0,02","0,01"
Q5QZ48,"0,55","0,24","0,11","0,07","0,16","0,03","0,01","0,01","0,08","0,00","0,02","0,01","0,01","0,08","0,20","0,01","0,01","0,01"
E1STN2,"0,11","0,07","0,02","0,06","0,12","0,02","0,01","0,02","0,06","0,00","0,01","0,01","0,01","0,04","0,06","0,01","0,01","0,01"
C4LEG2,"1,33","0,19","0,08","0,12","0,18","0,05","0,02","0,01","0,06","0,03","0,02","0,02","0,02","0,09","0,11","0,01","0,01","0,02"
Q15T03,"0,05","0,05","0,01","0,05","0,09","0,01","0,01","0,01","0,05","0,00","0,00","0,01","0,01","0,02","0,05","0,00","0,01","0,01"
Q482G6,"0,03","0,05","0,02","0,06","0,10","0,01","0,01","0,01","0,05","0,00","0,01","0,01","0,01","0,03","0,05","0,01","0,01","0,01"
Q3ILA3,"0,53","0,15","0,07","0,08","0,15","0,03","0,01","0,01","0,07","0,00","0,01","0,01","0,01","0,07","0,12","0,01","0,01","0,01"
Q12MU6,"0,13","0,08","0,02","0,06","0,10","0,02","0,01","0,01","0,05","0,00","0,01","0,01","0,01","0,03","0,06","0,01","0,01","0,01"


### Ajout des séquences

In [3]:
dict_sequences = {}

with open('FASTAs_TAMA.fasta', 'r') as sequences:
    lines = sequences.readlines()
    for j in range(len(lines)):
        if j%2 == 0: # Si c'est la ligne header
            code = re.search(r'\|([A-Z0-9]+)\|', lines[j]).group(1) # On extrait le code de l'enzyme
            if code in name_lines:
                dict_sequences[code] = ''
        else: # Si c'est la ligne avec le code protéique
            if code in name_lines:
                dict_sequences[code] = lines[j].strip() # On l'ajoute au dictionaire


### Normaliser avec Bradford

In [4]:
bradford = False

for line in range(len(df.index)):
    for row in range(len(df.columns)):
        n_line = df.index[line]
        n_row = df.columns[row]
        cell = df[n_row][n_line]
        value = float(cell.replace(',', '.'))
        
        if bradford: # On normalise si 'bradford' est à True
            value = value / bradford[n_line] 
    
        if value<0.0:  # On enlève les valeurs négatives
            df[n_row][n_line] = 0.0
        else:
            df[n_row][n_line] = value

df.head(50)


,2OATA,Pyr,2OPAA,2OHAA,4MT2OB,MOPA,tMePyr,2O2PAA,2O4PBA,HPA,3H2OBA,BOHiPr,BOHPh,4H2OA,4H4PA,2KDGA,"3,3diMe",NoCeto
C7RAM1,0.8,0.7,0.03,0.1,0.12,0.02,0.02,0.01,0.06,0.0,0.02,0.01,0.01,0.09,0.36,0.01,0.01,0.01
P44336,2.72,0.52,0.13,0.1,0.35,0.03,0.01,0.01,0.07,0.0,0.03,0.01,0.01,0.14,0.26,0.01,0.01,0.01
Q7VLP0,1.1,0.91,0.24,0.21,0.39,0.06,0.01,0.01,0.08,0.0,0.03,0.02,0.02,0.19,0.54,0.01,0.02,0.01
Q5QZ48,0.55,0.24,0.11,0.07,0.16,0.03,0.01,0.01,0.08,0.0,0.02,0.01,0.01,0.08,0.2,0.01,0.01,0.01
E1STN2,0.11,0.07,0.02,0.06,0.12,0.02,0.01,0.02,0.06,0.0,0.01,0.01,0.01,0.04,0.06,0.01,0.01,0.01
C4LEG2,1.33,0.19,0.08,0.12,0.18,0.05,0.02,0.01,0.06,0.03,0.02,0.02,0.02,0.09,0.11,0.01,0.01,0.02
Q15T03,0.05,0.05,0.01,0.05,0.09,0.01,0.01,0.01,0.05,0.0,0.0,0.01,0.01,0.02,0.05,0.0,0.01,0.01
Q482G6,0.03,0.05,0.02,0.06,0.1,0.01,0.01,0.01,0.05,0.0,0.01,0.01,0.01,0.03,0.05,0.01,0.01,0.01
Q3ILA3,0.53,0.15,0.07,0.08,0.15,0.03,0.01,0.01,0.07,0.0,0.01,0.01,0.01,0.07,0.12,0.01,0.01,0.01
Q12MU6,0.13,0.08,0.02,0.06,0.1,0.02,0.01,0.01,0.05,0.0,0.01,0.01,0.01,0.03,0.06,0.01,0.01,0.01


In [5]:
def get_targets():
    dic_solvants = {i:df.columns[i] for i in range(len(df.columns))}
    dic_enzymes = {i:(df.index)[i] for i in range(len(df.index))}
    targets = []
    for x in df.index:
        line = np.array(df.loc[x].values)
        targets.append(line)

    targets = np.array(targets).astype('float32')
    #print(dic_solvants)
    #print(dic_enzymes)
    #print("Targets shape :",targets.shape)
    #print()
    
    
    return targets, dic_solvants, dic_enzymes

### Enlever les lignes inutiles du fichier FASTA final

In [6]:
# On enlève les enzymes dont on s'occupe pas)

with open('FASTAs_TAMA.fasta', 'r') as sequences:
    with open('FASTAs_Final.fasta', 'w') as final_sequences: # Fichier final
        lines = sequences.readlines()
        for j in range(len(lines)):
            if j%2 == 0:
                code = re.search(r'\|([A-Z0-9]+)\|', lines[j]).group(1)
                if code in name_lines: # On ajoute la ligne au fichier comme dans l'original
                    final_sequences.write(lines[j])
                    final_sequences.write(lines[j+1])

### Alignement des données

In [7]:
# Pour pas perdre de temps à rerun à chaque fois
import json
change_final_dict = False # Est-ce qu'on veut run à nouveau l'executable ou juste load le dictionnaire sauvegardé

def get_aligned_sequences():
    if change_final_dict == False:
        with open("save.json", "r") as json_file:
            dict_aligned_seq = json.load(json_file)
    return dict_aligned_seq

In [8]:
# Le format du fichier est multiline_fasta

from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq
from Bio.Align.Applications import ClustalwCommandline

In [9]:
if change_final_dict == True: # Si on veut relancer l'executable pour refaire les calculs
    
    clustalw_exe = "clustalw2.exe" # L'emplacement que l'executable qui va faire le traitement des données
    output_file = "output.aln"
    input_file = "FASTAs_Final.fasta"

    sequences = list(SeqIO.parse(input_file, "fasta"))

    seq_records = [SeqRecord(seq.seq, id=seq.id) for seq in sequences] # Créer un objet SeqRecord pour chaque séquence

    temp_file = "temp.fasta" # Créer un fichier temporaire pour stocker les séquences à aligner
    SeqIO.write(seq_records, temp_file, "fasta")

    clustalw_cline = ClustalwCommandline( # Initialiser l'objet ClustalwCommandline pour traiter correctement les données
        cmd=clustalw_exe,
        infile=input_file,
        outfile=output_file,
        output="clustal",  # Format de sortie
        align=True,  # Effectuer un alignement complet
    )

    clustalw_cline() # Exécution de la commande ClustalW2

    aligned_sequences = list(SeqIO.parse(output_file, "clustal")) # Maintenant on peut lire les séquences alignées

    for aligned_seq in aligned_sequences:
        # Accéder à l'identifiant de la séquence
        seq_id = aligned_seq.id
        #print("ID de la séquence :", seq_id)

        # Accéder à la séquence elle-même
        sequence = aligned_seq.seq
        #print("Séquence :", sequence)

    # On crée à nouveau un dictionnaire avec les nouvelles séquences alignées
    dict_aligned_seq = {}

    for aligned_seq in aligned_sequences:

        code = re.search(r'\|([A-Z0-9]+)\|', aligned_seq.id).group(1)
        sequence = aligned_seq.seq
        dict_aligned_seq[code] = str(sequence)

    save_file = "save.json"

    with open(save_file, 'w') as s_file:
        json.dump(dict_aligned_seq, s_file) # Sauvegarder le dictionnaire dans le fichier JSON
dict_aligned_seq = get_aligned_sequences() 
#print(dict_aligned_seq)